# Lab 1/2: Logistic regression, line search and second order methods


Let $A \in \mathbb{R}^{n \times d}$ and $b \in \{-1, 1\}^n$. 
The logistic regression estimator for classification is defined as:
$$ \min_{x \in \mathbb{R}^d} \sum_{i=1}^n \log (1 + e^{- b_i \ a_i^\top x}) $$

In [ ]:
import numpy as np
from numpy.linalg import norm

def make_data(n, d, corruption=0.1, seed=None):
    if seed is not None:
        np.random.seed(seed)
    A = np.random.randn(n, d)
    x_true = np.random.randn(d)
    
    b = np.sign(A @ x_true)
    switch = np.random.choice(n, int(corruption*n), replace=False)
    b[switch] *= - 1
    return A, b

0. What does the above function do? To what quantity does each parameter correspond to?

In [ ]:
A, b = make_data(200, 100)  # keep those numbers

1. Define a function computing the gradient of the logistic regression objective function at ``x``. Check numerically that it is correct with ``scipy.optimize.check_grad``

In [ ]:
from scipy.optimize import check_grad


def logistic_gradient(x):
    # TODO
    # your code to compute the gradient of the logistic regression objective
    return

def logistic_value(x):
    # TODO
    # your code to compute the value of the logistic regression objective
    return


# check for a random vector of coefficient x0
x0 = np.random.randn(A.shape[1])
check_grad(logistic_value, logistic_gradient, x0)  # it must be small if your implementation is correct

2. What is the smoothness constant of the logistic loss? Is the logistic loss strongly convex?

3. Implement gradient descent on the objective with fixed stepsize $1/L$. Plot the convergence rate in objective. Does it match your expectations? Compare your solutions with scikit-learn's.


Add an option to use line search (using ``scipy.optimize.line_search``). Compare the convergence speed and comment.

In [ ]:
from scipy.optimize import line_search


def solve_with_GD(max_iter=500, use_line_search=False):
    objectives = []
    x = np.zeros(A.shape[1])
    
    for i in range(max_iter):
        # TODO
        # your implementation of gradient descent
        objectives[i] = logistic_value(x)
        # END TODO
        
        # break early if no progress made:
        if i > 0 and objectives[i-1] - objectives[i] < 1e-8:
            print("early exit, delta objective:", objectives[i-1] - objectives[i])
            break
    return objectives


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty=None, fit_intercept=False, tol=1e-8).fit(A, b)

f_star = logistic_value(clf.coef_[0])

4. Define a function to compute the Hessian of the logistic loss.

In [ ]:
def logistic_hessian(x):
    # TODO
    # your code to compute the Hessian of the logistic regression objective
    return

5. Implement Newton method without line search on this function. Comment on the possible behavior.
Add line search to Newton method using ``scipy.optimize.line_search``. Compare the convergence speed in number of iterations with respect to gradient descent. Compare in terms of time too.

In [ ]:
def solve_newton(max_iter, use_line_search):
    # TODO
    # your implementation of Newton method
    return

Is the Logistic regression problem strongly convex? Implement an adequate version of Nesterov accelerated gradient on the objective.


Add a regularization term $\lambda \Vert x \Vert^2 / 2$ to the objective, taking $\lambda = 0.001 L$ for example. What is the conditioning of this problem? Use again an appropriate Nesterov method on this objective; compare the convergence speed.

Now we consider $\ell_1$ regularized Logistic regression, ie minimize the sum of the logistic regression loss
and $\lambda \Vert x \Vert_1$. Show that for $\lambda$ greater than $\Vert A^\top b \Vert\infty/2$, the solution of this problem is 0.

Implement proximal gradient descent.

Implement FISTA on the same problem. FISTA consists in replacing the $x$ update step in NAG by:
$$_{k+1} = \mathrm{prox}_{\gamma g}(y_k - \gamma \nabla f(y_k)$$
while the $y$ update step is the same as in NAG.
Compare the convergence speed with that of ISTA (aka proximal gradient).

### Bonus

6. Implement the Quasi-Newton method with SR1/Broyden formula seen in class and run it on the logistic loss. Compare with previous algorithms in terms of iterations.

In [ ]:
def solve_quasi_newton(max_iter, use_line_search):
    # TODO
    # your implementation of Quasi-Newton method
    return

7) Use `scipy.optimize.fmin_lbfgs_b` to solve logistic regression. Compare convergence speed in iterations and time with respect to GD and Newton method. 
Pass the following callback function to the solver to retrieve time and iterates.

In [ ]:
import time 
import scipy

def make_callback():
    # closure
    times = []
    iterates = []
    def callback(x):
        iterates.append(x.copy())
        times.append(time.perf_counter())
    return times, iterates, callback

times, iterates, callback = make_callback()
scipy.optimize.fmin_lbfgs_b(..., callback=callback)  # TODO
print(times)
print(iterates)

8. Compute the Fenchel Rockafellar dual problem of Logistic regression. Does strong duality hold for this problem? 
What is the relationship between the primal and the dual solutions (use first order optimality condition iv) and v))

9. For one of the solvers above (e.g. Newton), plot the objective suboptimality (computing the exact solution with scikit-learn first, for example), together with the duality gap across iterations (using a cleverly chosen dual point). Comment.